xgboost total points by Pierre

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [18]:
data = pd.read_csv("cleaned_merged_seasons.csv")

<ipython-input-18-ce1e41c9b2f4>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("cleaned_merged_seasons.csv")


In [19]:
# valeurs manquantes
data = data.dropna(subset=['team_x'])

# Normaliser les caractéristiques
features_to_normalize = ["creativity", "ict_index", "influence", "threat"]
scaler = MinMaxScaler()
data[features_to_normalize] = scaler.fit_transform(data[features_to_normalize])

# Encode
data_encoded = pd.get_dummies(data, columns=["season_x", "position", "team_x", "kickoff_time", "opp_team_name"], drop_first=True)

# data type int
data_encoded['was_home'] = data_encoded['was_home'].astype(int)

# Retirer la colonne "name"
player_names = data_encoded['name'].reset_index(drop=True)
data_encoded = data_encoded.drop('name', axis=1)

<ipython-input-19-b453d90800f5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[features_to_normalize] = scaler.fit_transform(data[features_to_normalize])


In [20]:
print(player_names)

0                  Aaron Connolly
1                 Aaron Cresswell
2                      Aaron Mooy
3                  Aaron Ramsdale
4              Abdoulaye Doucouré
                   ...           
76312                Oliver Skipp
76313              Ryan Sessegnon
76314                Ashley Young
76315    Jeremy Sarmiento Morante
76316              Philip Billing
Name: name, Length: 76317, dtype: object


In [21]:
# Séparation des données en entrée (X) et sortie (y)
X = data_encoded.drop('total_points', axis=1)
y = data_encoded['total_points']

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test, player_names_train, player_names_test = train_test_split(
    X, y, player_names, test_size=0.2, random_state=42
)

# Redimensionnement des données pour être utilisées dans un modèle LSTM
X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))


In [22]:
# Création du modèle LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [23]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
1908/1908 [==============================] - 15s 7ms/step - loss: 5.5083 - val_loss: 5.3532
Epoch 2/50
1908/1908 [==============================] - 13s 7ms/step - loss: 5.2582 - val_loss: 5.3159
Epoch 3/50
1908/1908 [==============================] - 12s 6ms/step - loss: 5.1956 - val_loss: 5.4176
Epoch 4/50
1908/1908 [==============================] - 12s 6ms/step - loss: 5.1853 - val_loss: 5.0107
Epoch 5/50
1908/1908 [==============================] - 12s 6ms/step - loss: 5.2332 - val_loss: 5.2822
Epoch 6/50
1908/1908 [==============================] - 13s 7ms/step - loss: 5.2136 - val_loss: 5.2180
Epoch 7/50
1908/1908 [==============================] - 12s 7ms/step - loss: 5.2080 - val_loss: 5.4589
Epoch 8/50
1908/1908 [==============================] - 15s 8ms/step - loss: 5.4791 - val_loss: 5.5180
Epoch 9/50
1908/1908 [==============================] - 12s 7ms/step - loss: 5.2247 - val_loss: 5.2728
Epoch 10/50
1908/1908 [==============================] - 12s 7ms/step - l

In [24]:
# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

477/477 [==============================] - 2s 3ms/step
Mean Squared Error: 5.623644994583888
Mean Absolute Error: 1.531584656697774
R^2 Score: 0.04736971368733667
